# Mapping with Folium

The folium library provides an easy way to plot data handled in python into a Leaflet.js map


In [2]:
# folium imports
import folium
from folium.plugins import FloatImage
from folium.plugins.marker_cluster import MarkerCluster
from folium.plugins.heat_map_withtime import HeatMapWithTime
from folium import IFrame

# other imports
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.distance import great_circle, vincenty
import os
import csv 
import ast
from ipywidgets import interact
import base64
import random

folium.__version__

u'0.5.0'

# 1. Basic functionality of Folium

In [3]:
# instance of Nominatim, which is a geocoder for OSM servers and queries OSM data by name/address
geolocator = Nominatim()
born_place = geolocator.geocode("Pano Polemidia")

# make a list of containing lat, lon as floats
born_lat_lon = [born_place.latitude, born_place.longitude]

# create an instance of a folium map centered at my home in Cyprus
m = folium.Map(location=born_lat_lon, zoom_start=10)

# add default marker to the map with popup string
folium.Marker((born_place.latitude, born_place.longitude), popup = "This is where I was born").add_to(m)

# show the map
m

In [4]:
# Adding an image on the map

FloatImage("http://cyprus-mail.com/wp-content/uploads/2015/05/halloumi-370x223.jpg", 
           bottom=5, 
           left=60).add_to(m)

m

In [5]:
# Add further markers by clicking on the map -> easy access to surrounding coordinates
# If you double-click,they get removed

folium.features.ClickForMarker().add_to(m)
m

In [6]:
# add polylines to represent a trip

# my favourite beach
beach_place = geolocator.geocode("Kourion ")
beach_lat_lon = [beach_place.latitude, beach_place.longitude]

# create new map object
f = folium.Map(location=beach_lat_lon, zoom_start=12)

# make a list of [lat, lon] that describe a trip
trip_to_beach = [born_lat_lon] + [beach_lat_lon]

# add the polyline to the map
polyline=folium.PolyLine(trip_to_beach, color="red", popup="It takes approximately 15 minutes to go to the beach")
polyline.add_to(f)

# add marker to the start point
folium.RegularPolygonMarker(born_lat_lon, popup = "This is where I was born").add_to(f)

# add a marker to show where a really nice beach is
# popups can take HTML as well to highlight something interesting
folium.RegularPolygonMarker(beach_lat_lon, 
                            number_of_sides=6, 
                            color="green", 
                            fill_color="red", 
                            fill_opacity=0.1,
                            popup = "<h1>This is my favourite beach in United Kingdom!!!</h1>").add_to(f)


f

In [7]:
# add an image as pop-up

kourio_image = "./data/kourio_small.jpg"
encoded = base64.b64encode(open(kourio_image, 'rb').read())


html = '<img src="data:image/jpg;base64,{}">'.format
iframe = IFrame(html(encoded), width=420, height=200)
popup_image = folium.Popup(iframe, max_width=2950)

# add the popup to the map, use a different marker
folium.CircleMarker(beach_lat_lon, 
                    radius = 20,
                    popup = popup_image).add_to(f)

f

# 2. Visualizing parsed Google Timeline

In [9]:
# exploring the google timeline for one month

def make_list_coordinates(dir_with_timelines):
    all_days = []
    for timeline in os.listdir(dir_with_timelines):
        day = []
        with open(os.path.join(dir_with_timelines, timeline), "r") as f:
            reader = csv.reader(f, delimiter=";")
            next(reader)
            for row in reader:
                day += ast.literal_eval(row[4]) # in csv, the list is represented as string -> convert back to list
            all_days.append(day)
    return all_days

In [10]:
all_days = make_list_coordinates("./data/google_timelines/chris/csv/")

In [11]:
# plot one random day

# tiles from several providers
tiles_list = ["openstreetmap", "stamentoner", "stamenterrain", "stamenwatercolor", "CartoDB dark_matter"]

@interact(tiles = tiles_list)
def plot_marker_cluster(tiles="CartoDB dark_matter"):
    map1 = folium.Map([52.5136124, 13.3636964], tiles=tiles, zoom_start=12)
    
    # create a marker_cluster object to add the events in a clustered fashion
    marker_cluster = MarkerCluster().add_to(map1)

    # get a day randomly
    random_day = random.choice(all_days)
    for coord in random_day:
        folium.Marker(coord, popup=str(coord)).add_to(marker_cluster)
    
    return map1

In [12]:
# make a heatmap with time dimension for these days to see my pattern of movement over a longer period of time

@interact(radius=[15, 50])
def make_heat_map_with_time(radius = 30):
    # create a fresh map instance
    m = folium.Map([52.5136124, 13.3636964], tiles='stamentoner', zoom_start=12)

    # pass list of list of lists to heat map with time dimension
    # [[[x00, y00], [x01, y01]], ..., [[xn0, yn0], [xn1, yn1]] ]
    hm = HeatMapWithTime(all_days,
                    min_opacity = 0.1,
                    max_opacity = 0.5, 
                    radius = radius # radius around points for the heatmap
                    )
    hm.add_to(m)

    return m

# 3. Visualizing data from other sources

In [13]:
# plot a geojson 
s = folium.Map(location=[52.5, 13.4], zoom_start=9)
folium.GeoJson("./data/berliner-bezirke.geojson", name="Bezirke").add_to(s)

# add layer control
folium.LayerControl().add_to(s)
s

In [14]:
# geojson using choropleth
# choropleth applies a geojson overlay to a base map

x = folium.Map(location=[52.5, 13.4], zoom_start=9)
x.choropleth("./data/berliner-bezirke.geojson", 
             name="Bezirke", 
             fill_opacity=0.5, 
             line_opacity=0.5,
             fill_color="green")

# layer control
folium.LayerControl().add_to(x)
x

In [15]:
# read in the csv with pandas
df_bezirke_data = pd.read_csv("./data/bezirke_population_area_density.txt", 
                              header = None,
                              names = ["Bezirke", "Population", "Area", "Density"],
                              delimiter = "\t"
                             )
df_bezirke_data

,Bezirke,Population,Area,Density
0,Charlottenburg-Wilmersdorf,319.628,64.72,4.878
1,Friedrichshain-Kreuzberg,268.225,20.16,13.187
2,Lichtenberg,259.881,52.29,4.952
3,Marzahn-Hellersdorf,248.264,61.74,4.046
4,Mitte,332.919,39.47,8.272
5,Neukölln,310.283,44.93,6.804
6,Pankow,366.441,103.01,3.476
7,Reinickendorf,240.454,89.46,2.712
8,Spandau,223.962,91.91,2.441
9,Steglitz-Zehlendorf,293.989,102.50,2.818


In [16]:
mapa = folium.Map(location=[52.5, 13.4], zoom_start=10)

# add the information about the population of the zones in the plot
mapa.choropleth(geo_data = "./data/berliner-bezirke.geojson", 
                name="Bezirke with Population",
                data = df_bezirke_data,
                fill_color = "YlOrRd",
                fill_opacity = 0.6,
                line_opacity = 0.2,
                columns = ["Bezirke", "Population"],
                key_on = "feature.properties.spatial_alias", # where to bind the data to in the geojson
                legend_name = "Population in 1000s",
                highlight = True
                )

# add the information about the population density of the zones in the plot
mapa.choropleth(geo_data = "./data/berliner-bezirke.geojson", 
                name="Bezirke with Density",
                data = df_bezirke_data,
                fill_color = "PuRd",
                fill_opacity = 0.6,
                line_opacity = 0.2,
                columns = ["Bezirke", "Density"],
                key_on = "feature.properties.spatial_alias",
                legend_name = "Density in m^-2",
                highlight = True
                )

folium.LayerControl().add_to(mapa)
mapa


In [17]:
# save the map as an html you can share

mapa.save("./mapa.html")

# Sources

### Data
https://github.com/m-hoerz/berlin-shapes    
https://www.suche-postleitzahl.org/   
my data from google timeline

### Documentation
__Folium__: https://python-visualization.github.io/folium/      
__Leaflet.js__: http://leafletjs.com/reference-1.2.0.html 

### Other
http://comet.lehman.cuny.edu/owen/teaching/datasci/choroplethLab.html    
https://gis.stackexchange.com/questions/185897/how-can-i-include-html-in-a-folium-marker-popup

# THANK YOU

## Say hi @chr7stos on twitter